### 発話の例


In [34]:
import spacy
import re
import warnings
import ginza
import numpy as np
from collections import defaultdict
from collections import deque
nlp = spacy.load('ja_ginza')


In [35]:
doc = nlp('俺は彼が仲間だと信じてるよ。彼は人狼じゃないと思うよ。')
# 文節の係り受け解析
for span in ginza.bunsetu_spans(doc):
    for token in span.lefts:
        print(str(ginza.bunsetu_span(token))+' → '+str(span))
#spacy.displacy.render(ginza.bunsetu_spans(doc), style="dep", options={"compact":True})


彼が → 仲間だと
俺は → 信じてるよ。
仲間だと → 信じてるよ。
彼は → 人狼じゃないと
人狼じゃないと → 思うよ。


In [44]:
def show_nlp(sentence):
    doc = nlp(sentence)
    for sent in doc.sents:
        print(sent)
        for token in doc:
            print(token.i, token.orth_, token.lemma_, token.pos_, token.tag_, token.dep_, token.head.i)
        print('EOS')
    spacy.displacy.render(doc, style="dep", options={"compact":True})

s1 = "僕の役職は占い師ではありません"
show_nlp(s1)

僕の役職は占い師ではありません
0 僕 僕 PRON 代名詞 nmod 2
1 の の ADP 助詞-格助詞 case 0
2 役職 役職 NOUN 名詞-普通名詞-一般 nsubj 4
3 は は ADP 助詞-係助詞 case 2
4 占い師 占い師 NOUN 名詞-普通名詞-一般 ROOT 4
5 で で AUX 助詞-格助詞 cop 4
6 は は ADP 助詞-係助詞 fixed 5
7 あり ある VERB 動詞-非自立可能 fixed 5
8 ませ ます AUX 助動詞 fixed 5
9 ん ぬ AUX 助動詞 fixed 5
EOS


In [42]:
class Node:
    def __init__(self, id, token, parent):
        # id: 文中の語のid
        # word: 単語(lemma)
        # parent: 親ノードを表現．(id, <relation>)の．例えば(0, case)
        # isRoot: 根ノードか
        self.id = id
        self.token = token
        self.parent = parent
        self.child = []
        self.isRoot = (parent is None)

    def add_child(self, id, relation):
        # 子ノード 情報の追加．表記方法はparentと同じ．
        self.child.append((id, relation))

class SentenceTree: # 一つの文のみを処理する木
    def __init__(self, sentence, speaker):
        doc = nlp(sentence)
        self.id_lemma_dict = dict() # idとlemma
        self.node_list = [] # ノードを格納するリスト
        self.speaker = speaker # 発言者

        # ノードの登録
        for token in doc:
            self.id_lemma_dict[token.i] = token.lemma_
            idx = token.i
            lemma = token.lemma_

            if token.head.i == token.i: # ROOT
                parent = None
            else:
                parent = (token.head.i, token.dep_)
            self.node_list.append(Node(idx, lemma, parent))
        
        # 子ノード情報の登録
        for i in range(len(self.node_list)):
            if self.node_list[i].isRoot: # ROOTのインデックスを登録
                self.root_idx = i
            else: # ROOTでなければ親にアクセス
                (parent_id, relation) = self.node_list[i].parent
                self.node_list[parent_id].add_child(i, relation) # 子の追加
        # トークン数
        self.N = len(self.node_list)

        # 結果の確認
    
        for i in range(self.N):
            print(self.node_list[i].id, self.node_list[i].token, self.node_list[i].child)



    def bfs(self, token_list, start_idx=None, include_root=True):
        """
        input: token(捜索対象候補) start_idx(部分木の根，省略時はROOT) include_root(根も探索の対象か？)
        output: 存在すればid，しなければNone
        """
        if start_idx is None:
            start_idx = self.root_idx
        d = np.full(self.N, -1)
        q = deque()
        q.append(start_idx)
        d[start_idx] = 0

        if include_root and self.node_list[start_idx].token in token_list:
            return start_idx

        while len(q) != 0:
            n = q.popleft()
            for i in range(len(self.node_list[n].child)):
                child_id, _ =  self.node_list[n].child[i]
                if d[child_id] == -1:
                    q.append(child_id)
                    d[child_id] = d[n] + 1
                    if self.node_list[child_id].token in token_list:
                        return child_id
        return None

    def bfs_count(self, token_list, start_idx=None, include_root=True):
        """
        input: token(捜索対象候補) start_idx(部分木の根，省略時はROOT) include_root(根も探索の対象か？)
        output: 条件を満たす物の数を返す
        """ 
        ret = 0 
        if start_idx is None:
            start_idx = self.root_idx
        d = np.full(self.N, -1)
        q = deque()
        q.append(start_idx)
        d[start_idx] = 0

        if include_root and self.node_list[start_idx].token in token_list:
            ret += 1

        while len(q) != 0:
            n = q.popleft()
            for i in range(len(self.node_list[n].child)):
                child_id, _ =  self.node_list[n].child[i]
                if d[child_id] == -1:
                    q.append(child_id)
                    d[child_id] = d[n] + 1
                    if self.node_list[child_id].token in token_list:
                        ret += 1
        return ret


    def isCO(self):
        # CO判定のデモンストレーション
        positive = 1
        not_target = "not COMINGOUT/ESTIMATE"

        role_seer_list = ["占い師"]
        role_villager_list = ["村人", "むらびと", "一般人"]
        role_wolf_list = ["人狼", "狼", "黒"]
        role_possessed_list = ["狂人"]
        name_list = ["一郎", "二郎", "三郎"]
        first_person = ["私", "僕", "俺"]
        negative_aux_list = ["ぬ", "ない"]

        role_list = role_seer_list+role_villager_list+role_wolf_list+role_possessed_list
        person_list = name_list+first_person
        person_and_role = role_list+person_list
        
        n1 = None
        cop = None
        n2 = self.bfs(person_and_role)
        if n2 is None:
            return not_target+": N2発見できず"
        else:
            #print(self.id_lemma_dict[n2])

            for i in range(len(self.node_list[n2].child)):
                idx, relation = self.node_list[n2].child[i]
                if relation == "nsubj" and n1 is None: # 主題
                    #print(self.id_lemma_dict[idx])
                    n1 = self.bfs(person_and_role, start_idx=idx)
                elif relation == "cop": # 格助詞
                    cop = idx
            if cop is not None: # 念のため
                positive = (-1)**self.bfs_count(negative_aux_list, start_idx=cop) # negative auxの探索
            if n1 is None:
                return not_target+": N1発見できず"
        
        n1_token = self.id_lemma_dict[n1]
        n2_token = self.id_lemma_dict[n2]

        if (n1_token in role_list and n2_token in role_list) or\
            (n1_token in person_list and n2_token in person_list):
            return not_target+": 無意味な文" # "私は一郎だ"，"占い師は白だ"などの無意味な文
        
        else:
            if n1_token in role_list:
                role_token = n1_token
                person_token = n2_token
            else:
                role_token = n2_token
                person_token = n1_token

            if role_token in role_seer_list:
                role = "SEER"
            elif role_token in role_villager_list:
                role = "VILLAGER"
            elif role_token in role_possessed_list:
                role = "POSSESSED"
            else:
                role = "WEREWOLF"

            if person_token in first_person or person_token == self.speaker:
                person = self.speaker
                conversation_type = "COMINGOUT"
            else:
                person = person_token
                conversation_type = "ESTIMATE"

            if positive == 1:
                positive = "POSITIVE"
            else:
                positive = "NEGATIVE"

            return(self.speaker, conversation_type, person, role, positive)

s = "二郎は占い師ではありません。"
tree = SentenceTree(s, speaker="一郎")
tree.isCO()

('一郎', 'ESTIMATE', '二郎', 'SEER', 'NEGATIVE')

In [43]:
sentence_list = [
    "僕は占い師です。",
    "僕は人狼ではありません。",
    "二郎が占い師だというのは嘘だ。",
    "二郎が占い師だというのは嘘で、僕が占い師です",
    "二郎は狂人だと思う。",
    "僕は三郎が人狼だと思う。",
    "一郎は村人です。",
    "僕は一郎です。",
    "人狼は黒です。",
    "僕の役職は占い師です。",
    "三郎の役職は狂人だと僕は思う。",
    "俺は三郎が村人だって信じてるよ。",
    "三郎が村人だと僕は信じているよ。",
    "二郎は人狼に違いない。",
    "二郎の発言には矛盾があるから、二郎は狂人に違いない"
]
with open("result/res_co.txt", encoding="utf-8", mode="w") as f:
    for i in range(len(sentence_list)):
        tree = SentenceTree(sentence_list[i], speaker="一郎")
        f.write(sentence_list[i]+"\n")
        f.write(str(tree.isCO())+"\n")
        f.write("\n")
        print(sentence_list[i])
        print(tree.isCO(), "\n")


僕は占い師です。
('一郎', 'COMINGOUT', '一郎', 'SEER', 'POSITIVE') 

僕は人狼ではありません。
('一郎', 'COMINGOUT', '一郎', 'WEREWOLF', 'NEGATIVE') 

二郎が占い師だというのは嘘だ。
('一郎', 'ESTIMATE', '二郎', 'SEER', 'POSITIVE') 

二郎が占い師だというのは嘘で、僕が占い師です
('一郎', 'COMINGOUT', '一郎', 'SEER', 'POSITIVE') 

二郎は狂人だと思う。
('一郎', 'ESTIMATE', '二郎', 'POSSESSED', 'POSITIVE') 

僕は三郎が人狼だと思う。
('一郎', 'ESTIMATE', '三郎', 'WEREWOLF', 'POSITIVE') 

一郎は村人です。
('一郎', 'COMINGOUT', '一郎', 'VILLAGER', 'POSITIVE') 

僕は一郎です。
not COMINGOUT/ESTIMATE: 無意味な文 

人狼は黒です。
not COMINGOUT/ESTIMATE: 無意味な文 

僕の役職は占い師です。
('一郎', 'COMINGOUT', '一郎', 'SEER', 'POSITIVE') 

三郎の役職は狂人だと僕は思う。
('一郎', 'ESTIMATE', '三郎', 'POSSESSED', 'POSITIVE') 

俺は三郎が村人だって信じてるよ。
not COMINGOUT/ESTIMATE: N1発見できず 

三郎が村人だと僕は信じているよ。
('一郎', 'ESTIMATE', '三郎', 'VILLAGER', 'POSITIVE') 

二郎は人狼に違いない。
('一郎', 'ESTIMATE', '二郎', 'WEREWOLF', 'POSITIVE') 

二郎の発言には矛盾があるから、二郎は狂人に違いない
('一郎', 'ESTIMATE', '二郎', 'POSSESSED', 'POSITIVE') 

